<a href="https://colab.research.google.com/github/OscarRojasG/OCR/blob/main/OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset

The dataset contains 26 folders (A-Z) containing handwritten images in size 28x28 pixels, each alphabet in the image is centre fitted to 20x20 pixel box.

Each image is stored as Gray-level

https://www.kaggle.com/datasets/sachinpatel21/az-handwritten-alphabets-in-csv-format/

## Cargar dataset completo

No recomendable... se agota la RAM de colab y tarda demasiado en cargar.

Se recomienda reducir el volumen del dataset (revisar sección "Exportar").

In [175]:
import pandas as pd
import numpy as np

df = pd.read_csv("A_Z Handwritten Data.csv", header=None, dtype=np.uint8, nrows=10000)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Cargar muestra

In [177]:
import pandas as pd
import numpy as np

df = pd.read_csv("sample.csv", dtype=np.uint8, header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [325]:
from PIL import Image as Img

pixels = df.iloc[0].to_numpy()[1:]
pixels = np.reshape(pixels, (28, 28))

img = Img.fromarray(pixels)
img

In [326]:
def draw_image(pixel_arr):
    pixels = np.reshape(pixel_arr, (28, 28))
    img = Img.fromarray(pixels)
    return img

## Exportar

Funciones para exportar dataset con 1000 filas por clase.

In [49]:
letters = np.zeros(shape=(26, 1000, 785))

In [188]:
# Asegurarse que todas las clases tengan 1000 datos como mínimo
df.iloc[:, 0].value_counts()

0     1000
1     1000
24    1000
23    1000
22    1000
21    1000
20    1000
19    1000
18    1000
17    1000
16    1000
15    1000
14    1000
13    1000
12    1000
11    1000
10    1000
9     1000
8     1000
7     1000
6     1000
5     1000
4     1000
3     1000
2     1000
25    1000
Name: 0, dtype: int64

In [169]:
letter = 25 # Recorrer de 0 a 25

arr = df[df.iloc[:, 0] == letter].to_numpy()
arr = arr[:1000]
letters[letter] = arr

In [355]:
flt_arr = letters.reshape(26000, 785)
np.savetxt("sample.csv", flt_arr, delimiter=",")

# Modelo

In [207]:
from sklearn.model_selection import train_test_split

Y = df.iloc[:, 0].values
X = df.iloc[:, 1:].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1, stratify=Y)

Creación del modelo

In [351]:
from keras import models
from keras import layers

model = models.Sequential()

model.add(layers.Dense(100, activation='sigmoid', input_shape=(784,)))
model.add(layers.Dense(26, activation='sigmoid'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Entrenamiento

In [352]:
model.fit(X_train, Y_train, epochs=100)

Epoch 1/100
650/650 [==============================] - 2s 3ms/step - loss: 1.9050 - accuracy: 0.5247
Epoch 2/100
650/650 [==============================] - 2s 3ms/step - loss: 1.0713 - accuracy: 0.7307
Epoch 3/100
650/650 [==============================] - 2s 3ms/step - loss: 0.8869 - accuracy: 0.7620
Epoch 4/100
650/650 [==============================] - 2s 4ms/step - loss: 0.8141 - accuracy: 0.7772
Epoch 5/100
650/650 [==============================] - 3s 4ms/step - loss: 0.7620 - accuracy: 0.7921
Epoch 6/100
650/650 [==============================] - 2s 3ms/step - loss: 0.7136 - accuracy: 0.7981
Epoch 7/100
650/650 [==============================] - 2s 3ms/step - loss: 0.6851 - accuracy: 0.8071
Epoch 8/100
650/650 [==============================] - 2s 3ms/step - loss: 0.6670 - accuracy: 0.8109
Epoch 9/100
650/650 [==============================] - 2s 3ms/step - loss: 0.6655 - accuracy: 0.8091
Epoch 10/100
650/650 [==============================] - 2s 3ms/step - loss: 0.6493 - accura

Validación con datos de prueba: entropía categórica - accuracy

In [353]:
model.evaluate(X_test, Y_test, verbose=False)

[0.4517442584037781, 0.8763461709022522]

In [336]:
def predict(test_set, idx):
    prob = model.predict(test_set[idx:idx+1])
    pred = prob.argmax()
    letter = chr(ord('A') + pred)

    print(letter)
    return draw_image(test_set[idx])

In [354]:
from io import BytesIO
from IPython.display import display, Image

for i in range(10):
    img = predict(X_test, i)
    bio = BytesIO()
    img.save(bio, format='png')
    display(Image(bio.getvalue(), format='png'))

1/1 [==============================] - 0s 49ms/step
S


1/1 [==============================] - 0s 20ms/step
O


1/1 [==============================] - 0s 21ms/step
M


1/1 [==============================] - 0s 21ms/step
M


1/1 [==============================] - 0s 21ms/step
U


1/1 [==============================] - 0s 24ms/step
M


1/1 [==============================] - 0s 21ms/step
S


1/1 [==============================] - 0s 24ms/step
K


1/1 [==============================] - 0s 20ms/step
D


1/1 [==============================] - 0s 21ms/step
E
